In [2]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import trompy as tp
import csv
from datetime import datetime, timedelta


# the next two lines will expand the printed tables
pd.options.display.max_rows= 40000
pd.options.display.max_columns= 40000


ctl_color= "gray"
exp_color="deepskyblue"
fig_folder=r"../plots\\"
READ_METAFILE= pd.read_excel(r"/workspaces/FEDPROFERENCE/FEDXD_METAFILE.xls" ,sheet_name="METAFILE" )

READ_METAFILE


,MOUSEID,SEX,BATCH,FEDFILE,CHOICE_SESSION,FED_PELLET,MODE,DIET
0,FEDXDF1,F,1,FED001_032923_02.CSV,0,MIX,FF,PR
1,FEDXDF1,F,1,FED001_033023_01.CSV,0,MIX,FR,PR
2,FEDXDF1,F,1,FED001_040923_00.CSV,1,CAS20,FR,PR
3,FEDXDF1,F,1,FED002_040923_00.CSV,1,CAS5,FR,PR
4,FEDXDF1,F,1,FED001_041323_00.CSV,2,CAS20,FR,PR
5,FEDXDF1,F,1,FED002_041323_00.CSV,2,SUC,FR,PR
6,FEDXDF1,F,1,FED001_041723_00.CSV,3,CAS5,FR,PR
7,FEDXDF1,F,1,FED002_041723_00.CSV,3,SUC,FR,PR
8,FEDXDF1,F,1,FED001_042123_03.CSV,4,CAS5,CE,PR
9,FEDXDF1,F,1,FED002_042123_02.CSV,4,SUC,CE,PR


In [3]:
metafile_path = "../FEDXD_METAFILE.xls"  # Adjusted path to the uploaded file
rows, header = tp.metafilereader(metafile_path, sheetname="METAFILE")

def get_FEDevents(filename, eventname):
    formats = ['%m/%d/%Y %H:%M:%S']  # Adjusted to the format in your CSV
    file = open(filename)
    csvreader = csv.reader(file)
    next(csvreader)  # Skip header
    rows = [row for row in csvreader]

    if not rows:  # Check if rows is empty
        print(f"No data in file {filename}.")
        return []

    # Assuming the first column contains the date and time info
    # and the "Event" column is consistent with the provided files
    event_column_index = 9  # Based on your CSV structure
    
    # Determine which date format should be used (assuming consistency, so just using one format)
    date_format = formats[0]

    timestamps = []
    for row in rows:
        if row[event_column_index] == eventname:
            date_time_obj = datetime.strptime(row[0], date_format)
            timestamps.append(date_time_obj)

    if not timestamps:  # If no events match, return an empty list
        return []

    # Calculate time since "lights on", using the first event as reference if needed
    t0 = timestamps[0].strftime(date_format)
    day = t0.split()[0]
    lightson = day + " 07:00:00"  # Assuming lights on at 7 AM
    refpoint = datetime.strptime(lightson, date_format)

    pellettimes = [(t - refpoint).total_seconds() / 3600 for t in timestamps]

    return pellettimes
# Initialize a dictionary to hold lists for each unique group
grouped_lists = {}

# Iterate through the rows to process and group data
for row in rows:
    mouse_id = row[0]
    sex = row[1]  # Assuming this is the correct column for SEX
    diet = row[7]  # Assuming this is the correct column for DIET
    choice_session = row[4]
    pellet_type = row[5]
    filename = f"../data/{row[3]}"


    # Create a unique key for each group
    group_key = f"{sex}_{diet}_{choice_session}_{pellet_type}"
    
    # Collect timestamps for this row's file and pellet event
    timestamps = get_FEDevents(filename, "Pellet")
    pellet_count = len(timestamps)
    
    # Initialize the list for this group if it doesn't exist
    if group_key not in grouped_lists:
        grouped_lists[group_key] = []
    
    # Append the data for this mouse to the list for its group
    grouped_lists[group_key].append({
        "MOUSEID": mouse_id,
        "SEX": sex,
        "DIET": diet,
        "CHOICE_SESSION": choice_session,
        "PELLET_TYPE": pellet_type,
        "COUNT": pellet_count
    })

# Cell below gets Mega meal as a continuation of meals - i.e. mega meals are yet counted as "meal" however registered separately in a column
# min pellet for meal is 2 and as soon as pellet count goes above or equal 5 it is also marked as a mega meal

In [9]:


def get_FEDevents(filename, eventname, formats=['%m/%d/%Y %H:%M:%S']):
    with open(filename) as file:
        csvreader = csv.reader(file)
        next(csvreader)  # Skip header
        rows = [row for row in csvreader if row[9] == eventname]

    timestamps = [datetime.strptime(row[0], formats[0]) for row in rows]
    
    if not timestamps:
        return []

    refpoint = timestamps[0].replace(hour=7, minute=0, second=0, microsecond=0)
    pellettimes = [(t - refpoint).total_seconds() / 3600 for t in timestamps]

    return pellettimes, refpoint

def get_meal_and_snack_metrics(pellettimes, meal_threshold=1/60, min_pellets_for_meal=2):
    if not pellettimes:
        return (0, 0, 0, 0, 0, 0, [0]*24, 0, 0)

    IPIs = np.diff(np.array(pellettimes))
    meals = []
    snacks = []
    current_event = [pellettimes[0]]
    mega_meals = []

    for i, ipi in enumerate(IPIs):
        if ipi <= meal_threshold:
            current_event.append(pellettimes[i + 1])
        else:
            if len(current_event) >= min_pellets_for_meal:
                meals.append(current_event)
                if len(current_event) > 5:
                    mega_meals.append(len(current_event))
            else:
                snacks.append(current_event)
            current_event = [pellettimes[i + 1]]

    # Handle the last sequence
    if current_event:
        if len(current_event) >= min_pellets_for_meal:
            meals.append(current_event)
            if len(current_event) > 5:
                mega_meals.append(len(current_event))
        else:
            snacks.append(current_event)

    nmeals = len(meals)
    nsnacks = len(snacks)
    
    hourly_meals = np.zeros(24)
    for meal in meals:
        start_hour = int(meal[0]) % 24
        hourly_meals[start_hour] += 1

    total_pellets = len(pellettimes)
    mealsize = sum(len(meal) for meal in meals) / nmeals if nmeals else 0
    snack_size = sum(len(snack) for snack in snacks) / nsnacks if nsnacks else total_pellets / nsnacks if nsnacks else 0
    total_observation_period = max(pellettimes) - min(pellettimes)
    meal_frequency = nmeals / total_observation_period if total_observation_period > 0 else 0
    snack_frequency = nsnacks / total_observation_period if total_observation_period > 0 else 0
    mega_meal_count = len(mega_meals)
    average_mega_meal_size = sum(mega_meals) / mega_meal_count if mega_meal_count > 0 else 0

    return mealsize, snack_size, nmeals, meal_frequency, nsnacks, snack_frequency, hourly_meals.tolist(), mega_meal_count, average_mega_meal_size

# Assuming 'rows' is populated from the metafile as in your initial "read_data" script
grouped_lists = {}

for row in rows:
    mouse_id = row[0]
    sex = row[1]
    diet = row[7]
    choice_session = row[4]
    pellet_type = row[5]
    filename = f"../data/{row[3]}"

    group_key = f"{sex}_{diet}_{choice_session}_{pellet_type}"
    
    pellettimes, refpoint = get_FEDevents(filename, "Pellet")
    pellet_count = len(pellettimes)
    mealsize, snack_size, nmeals, meal_frequency, nsnacks, snack_frequency, hourly_meals, mega_meal_count, average_mega_meal_size = get_meal_and_snack_metrics(pellettimes)
    
    entry = {
        "MOUSEID": mouse_id,
        "SEX": sex,
        "DIET": diet,
        "CHOICE_SESSION": choice_session,
        "PELLET_TYPE": pellet_type,
        "COUNT": pellet_count,
        "MEAL_SIZE": mealsize,
        "SNACK_SIZE": snack_size,
        "NUMBER_OF_MEALS": nmeals,
        "MEAL_FREQUENCY": meal_frequency,
        "NUMBER_OF_SNACKS": nsnacks,
        "SNACK_FREQUENCY": snack_frequency,
        "NUMBER_OF_MEGA_MEALS": mega_meal_count,
        "AVERAGE_MEGA_MEAL_SIZE": average_mega_meal_size
    }
    
    # Add hourly meals data
    for hour in range(24):
        entry[f"MEALS_HOUR_{hour}"] = hourly_meals[hour]

    if group_key not in grouped_lists:
        grouped_lists[group_key] = []
    
    grouped_lists[group_key].append(entry)

# Convert grouped_lists to DataFrame
data_to_convert = [item for sublist in grouped_lists.values() for item in sublist]
df = pd.DataFrame(data_to_convert)

df  # Display the first few rows of the DataFrame to verify the structure


,MOUSEID,SEX,DIET,CHOICE_SESSION,PELLET_TYPE,COUNT,MEAL_SIZE,SNACK_SIZE,NUMBER_OF_MEALS,MEAL_FREQUENCY,NUMBER_OF_SNACKS,SNACK_FREQUENCY,NUMBER_OF_MEGA_MEALS,AVERAGE_MEGA_MEAL_SIZE,MEALS_HOUR_0,MEALS_HOUR_1,MEALS_HOUR_2,MEALS_HOUR_3,MEALS_HOUR_4,MEALS_HOUR_5,MEALS_HOUR_6,MEALS_HOUR_7,MEALS_HOUR_8,MEALS_HOUR_9,MEALS_HOUR_10,MEALS_HOUR_11,MEALS_HOUR_12,MEALS_HOUR_13,MEALS_HOUR_14,MEALS_HOUR_15,MEALS_HOUR_16,MEALS_HOUR_17,MEALS_HOUR_18,MEALS_HOUR_19,MEALS_HOUR_20,MEALS_HOUR_21,MEALS_HOUR_22,MEALS_HOUR_23
0,FEDXDF1,F,PR,0.0,MIX,210,4.666667,1.0,42,1.932688,14,0.644229,10,7.400000,2.0,1.0,0.0,3.0,3.0,1.0,1.0,4.0,0.0,0.0,1.0,3.0,1.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0
1,FEDXDF1,F,PR,0.0,MIX,296,3.271429,1.0,70,1.471112,67,1.408064,5,7.600000,7.0,3.0,5.0,0.0,0.0,3.0,2.0,0.0,2.0,1.0,1.0,2.0,4.0,1.0,5.0,4.0,8.0,5.0,5.0,4.0,0.0,0.0,6.0,2.0
2,FEDXDF2,F,PR,0.0,MIX,350,3.387097,1.0,93,4.203179,35,1.581841,10,6.900000,2.0,1.0,8.0,11.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,4.0,5.0,7.0,7.0,6.0,6.0,6.0,6.0,4.0,5.0,2.0
3,FEDXDF2,F,PR,0.0,MIX,330,3.011905,1.0,84,1.727763,77,1.583783,5,6.600000,6.0,2.0,3.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,5.0,4.0,8.0,8.0,7.0,9.0,7.0,4.0,4.0,3.0,4.0,3.0
4,FEDXDF3,F,PR,0.0,MIX,291,3.652174,1.0,69,2.923626,39,1.652484,8,7.625000,4.0,5.0,0.0,4.0,4.0,0.0,5.0,0.0,2.0,3.0,0.0,1.0,2.0,4.0,6.0,5.0,5.0,7.0,4.0,4.0,3.0,1.0,0.0,0.0
5,FEDXDF3,F,PR,0.0,MIX,375,4.144737,1.0,76,1.565764,60,1.236129,18,8.000000,5.0,0.0,5.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,2.0,1.0,6.0,5.0,6.0,6.0,7.0,6.0,4.0,4.0,4.0,3.0,1.0,4.0
6,FEDXDF4,F,PR,0.0,MIX,294,4.257576,1.0,66,2.779630,13,0.547503,16,7.250000,4.0,3.0,7.0,6.0,0.0,1.0,2.0,0.0,4.0,1.0,2.0,1.0,5.0,2.0,3.0,2.0,4.0,5.0,4.0,2.0,2.0,0.0,3.0,3.0
7,FEDXDF4,F,PR,0.0,MIX,312,4.928571,1.0,56,1.150724,36,0.739751,22,7.181818,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,4.0,4.0,4.0,7.0,3.0,4.0,4.0,4.0,4.0,1.0,0.0,4.0
8,FEDXDF5,F,PR,0.0,MIX,227,5.538462,1.0,39,2.372744,11,0.669235,19,8.000000,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0
9,FEDXDF5,F,PR,0.0,MIX,363,4.920635,1.0,63,1.362612,53,1.146325,24,8.416667,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,4.0,4.0,3.0,0.0,3.0


# Cell below gets Mega Meals separated from Meals and Snacks
# Snack contains 1 pellet, Meal contains 2-4 pellets and Mega Meal contains 5 or more pellets

In [15]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd

def get_FEDevents(filename, eventname, formats=['%m/%d/%Y %H:%M:%S']):
    with open(filename) as file:
        csvreader = csv.reader(file)
        next(csvreader)  # Skip header
        rows = [row for row in csvreader if row[9] == eventname]

    timestamps = [datetime.strptime(row[0], formats[0]) for row in rows]
    
    if not timestamps:
        return []

    refpoint = timestamps[0].replace(hour=7, minute=0, second=0, microsecond=0)
    pellettimes = [(t - refpoint).total_seconds() / 3600 for t in timestamps]

    return pellettimes, refpoint

def get_meal_and_snack_metrics(pellettimes, meal_threshold=1/60):
    if not pellettimes:
        return (0, 0, 0, 0, 0, 0, [0]*24, 0, 0)

    IPIs = np.diff(np.array(pellettimes))
    meals = []
    snacks = []
    mega_meals = []
    current_event = [pellettimes[0]]

    for i, ipi in enumerate(IPIs):
        if ipi <= meal_threshold:
            current_event.append(pellettimes[i + 1])
        else:
            if len(current_event) == 1:
                snacks.append(current_event)
            elif 2 <= len(current_event) <= 4:
                meals.append(current_event)
            elif len(current_event) >= 5:
                mega_meals.append(current_event)
            current_event = [pellettimes[i + 1]]

    # Handle the last sequence
    if current_event:
        if len(current_event) == 1:
            snacks.append(current_event)
        elif 2 <= len(current_event) <= 4:
            meals.append(current_event)
        elif len(current_event) >= 5:
            mega_meals.append(current_event)

    nmeals = len(meals)
    nsnacks = len(snacks)
    mega_meal_count = len(mega_meals)
    
    hourly_meals = np.zeros(24)
    for meal in meals:
        start_hour = int(meal[0]) % 24
        hourly_meals[start_hour] += 1

    total_pellets = len(pellettimes)
    mealsize = sum(len(meal) for meal in meals) / nmeals if nmeals else 0
    snack_size = sum(len(snack) for snack in snacks) / nsnacks if nsnacks else 0
    total_observation_period = max(pellettimes) - min(pellettimes)
    meal_frequency = nmeals / total_observation_period if total_observation_period > 0 else 0
    snack_frequency = nsnacks / total_observation_period if total_observation_period > 0 else 0
    average_mega_meal_size = sum(len(meal) for meal in mega_meals) / mega_meal_count if mega_meal_count else 0

    return mealsize, snack_size, nmeals, meal_frequency, nsnacks, snack_frequency, hourly_meals.tolist(), mega_meal_count, average_mega_meal_size

# Assuming 'rows' is populated from the metafile as in your initial "read_data" script
grouped_lists = {}

for row in rows:
    mouse_id = row[0]
    sex = row[1]
    diet = row[7]
    choice_session = row[4]
    pellet_type = row[5]
    filename = f"../data/{row[3]}"

    group_key = f"{sex}_{diet}_{choice_session}_{pellet_type}"
    
    pellettimes, refpoint = get_FEDevents(filename, "Pellet")
    pellet_count = len(pellettimes)
    mealsize, snack_size, nmeals, meal_frequency, nsnacks, snack_frequency, hourly_meals, mega_meal_count, average_mega_meal_size = get_meal_and_snack_metrics(pellettimes)
    
    entry = {
        "MOUSEID": mouse_id,
        "SEX": sex,
        "DIET": diet,
        "CHOICE_SESSION": choice_session,
        "PELLET_TYPE": pellet_type,
        "COUNT": pellet_count,
        "MEAL_SIZE": mealsize,
        "SNACK_SIZE": snack_size,
        "NUMBER_OF_MEALS": nmeals,
        "MEAL_FREQUENCY": meal_frequency,
        "NUMBER_OF_SNACKS": nsnacks,
        "SNACK_FREQUENCY": snack_frequency,
        "NUMBER_OF_MEGA_MEALS": mega_meal_count,
        "AVERAGE_MEGA_MEAL_SIZE": average_mega_meal_size
    }
    
    # Add hourly meals data
    for hour in range(24):
        entry[f"MEALS_HOUR_{hour}"] = hourly_meals[hour]

    if group_key not in grouped_lists:
        grouped_lists[group_key] = []
    
    grouped_lists[group_key].append(entry)

# Convert grouped_lists to DataFrame
data_to_convert = [item for sublist in grouped_lists.values() for item in sublist]
df = pd.DataFrame(data_to_convert)

df

,MOUSEID,SEX,DIET,CHOICE_SESSION,PELLET_TYPE,COUNT,MEAL_SIZE,SNACK_SIZE,NUMBER_OF_MEALS,MEAL_FREQUENCY,NUMBER_OF_SNACKS,SNACK_FREQUENCY,NUMBER_OF_MEGA_MEALS,AVERAGE_MEGA_MEAL_SIZE,MEALS_HOUR_0,MEALS_HOUR_1,MEALS_HOUR_2,MEALS_HOUR_3,MEALS_HOUR_4,MEALS_HOUR_5,MEALS_HOUR_6,MEALS_HOUR_7,MEALS_HOUR_8,MEALS_HOUR_9,MEALS_HOUR_10,MEALS_HOUR_11,MEALS_HOUR_12,MEALS_HOUR_13,MEALS_HOUR_14,MEALS_HOUR_15,MEALS_HOUR_16,MEALS_HOUR_17,MEALS_HOUR_18,MEALS_HOUR_19,MEALS_HOUR_20,MEALS_HOUR_21,MEALS_HOUR_22,MEALS_HOUR_23
0,FEDXDF1,F,PR,0.0,MIX,210,3.272727,1.0,22,1.012361,14,0.644229,20,6.200000,1.0,0.0,0.0,2.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,0.0,2.0
1,FEDXDF1,F,PR,0.0,MIX,296,2.607143,1.0,56,1.176890,67,1.408064,14,5.928571,6.0,2.0,5.0,0.0,0.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,4.0,4.0,7.0,3.0,4.0,4.0,0.0,0.0,5.0,2.0
2,FEDXDF2,F,PR,0.0,MIX,350,2.684932,1.0,73,3.299269,35,1.581841,20,5.950000,2.0,0.0,6.0,10.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,4.0,3.0,3.0,5.0,6.0,6.0,3.0,5.0,4.0,4.0,5.0,1.0
3,FEDXDF2,F,PR,0.0,MIX,330,2.602740,1.0,73,1.501508,77,1.583783,11,5.727273,6.0,2.0,3.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,5.0,1.0,7.0,8.0,6.0,7.0,6.0,3.0,4.0,3.0,4.0,2.0
4,FEDXDF3,F,PR,0.0,MIX,291,2.764706,1.0,51,2.160941,39,1.652484,18,6.166667,3.0,4.0,0.0,2.0,4.0,0.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,5.0,4.0,6.0,2.0,2.0,2.0,0.0,0.0,0.0
5,FEDXDF3,F,PR,0.0,MIX,375,2.754717,1.0,53,1.091914,60,1.236129,23,7.347826,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,5.0,3.0,6.0,2.0,7.0,3.0,2.0,3.0,4.0,2.0,0.0,2.0
6,FEDXDF4,F,PR,0.0,MIX,294,2.976190,1.0,42,1.768856,13,0.547503,24,6.500000,4.0,3.0,5.0,6.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,5.0,0.0,2.0,1.0,0.0,1.0,2.0,1.0,2.0,0.0,3.0,3.0
7,FEDXDF4,F,PR,0.0,MIX,312,3.074074,1.0,27,0.554814,36,0.739751,29,6.655172,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0
8,FEDXDF5,F,PR,0.0,MIX,227,3.105263,1.0,19,1.155952,11,0.669235,20,7.850000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,3.0,3.0,2.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0
9,FEDXDF5,F,PR,0.0,MIX,363,2.514286,1.0,35,0.757007,53,1.146325,28,7.928571,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,3.0,5.0,4.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0,2.0
